## Avila training
 This notebook loads the Avila dataset (https://archive.ics.uci.edu/ml/datasets/Avila), preprocesses it and trains a simple model

In [1]:
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))



Load data from CSV and save it to a suitable format. This can be skipped if concrete_data.npz is in assets.

In [ ]:
import pandas as pd
import numpy as np

def load_avila(path):
    df = pd.read_csv(path, delimiter=',', header=None)

    labels = df[10]
    x = df.drop(columns=10)

    possible_labels = labels.unique().tolist()
    print(possible_labels)
    print(len(possible_labels))
    y = labels.map(lambda x: possible_labels.index(x))

    return x.to_numpy(), y.to_numpy()
    
x_train, y_train = load_avila(os.path.join(PROJ_DIR, 'assets', 'data', 'avila-tr.txt'))
x_test, y_test = load_avila(os.path.join(PROJ_DIR, 'assets', 'data', 'avila-tr.txt'))

In [ ]:
# Save to assets
np.savez(os.path.join(PROJ_DIR, 'assets', 'data', 'avila'),\
        x_train=x_train,\
        x_test=x_test,\
        y_train=y_train,\
        y_test=y_test)

Load data from file

In [2]:
file_data = np.load(os.path.join(PROJ_DIR, 'assets', 'data', 'avila.npz'))
x_train = file_data['x_train']
x_test = file_data['x_test']
y_train = file_data['y_train']
y_test = file_data['y_test']

In [24]:
import torch
#import matplotlib.pyplot as plt


class RandomZeroMasker(torch.nn.Module):
    def forward(self, x): # Assumes inputs are (batch_size, num_vars)
        print(x.shape)
        selection_levels = torch.rand((x.shape[0], 1)) # A different selection level for each element of the batch
        selected_pixels = torch.le(torch.rand(x.shape), selection_levels) # A different selection level for each element of the batch
        print(selected_pixels.shape)
        print((x * selected_pixels).shape)
        return x * selected_pixels

# Test
masked_batch = RandomZeroMasker()(torch.tensor(x_train).float())

for i in range(4):#x_train[0]):
    #plt.imshow(np.moveaxis(masked_batch[i].numpy(), 0, -1), vmin=0.0, vmax=1.0, cmap="gray")
    print(x_train[i])
    print(masked_batch[i])
    print('//')

torch.Size([10430, 10])
torch.Size([10430, 10])
torch.Size([10430, 10])
[ 0.266074 -0.16562   0.32098   0.483299  0.17234   0.273364  0.371178
  0.929823  0.251173  0.159345]
tensor([ 0.2661, -0.1656,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.9298,
         0.0000,  0.0000])
//
[ 0.130292  0.870736 -3.210528  0.062493  0.261718  1.43606   1.46594
  0.636203  0.282354  0.515587]
tensor([0.0000, 0.0000, -0.0000, 0.0625, 0.0000, 1.4361, 0.0000, 0.0000, 0.0000,
        0.0000])
//
[-0.116585  0.069915  0.068476 -0.783147  0.261718  0.439463 -0.081827
 -0.888236 -0.123005  0.582939]
tensor([-0.0000,  0.0699,  0.0685, -0.7831,  0.2617,  0.4395, -0.0818, -0.8882,
        -0.0000,  0.5829])
//
[ 0.031541  0.2976   -3.210528 -0.58359  -0.721442 -0.307984  0.710932
  1.051693  0.594169 -0.533994]
tensor([ 0.0000,  0.2976, -3.2105, -0.5836, -0.7214, -0.3080,  0.7109,  1.0517,
         0.5942, -0.5340])
//


Train a MLP model

In [27]:
import torch

MODEL_NEURONS = 100
#MODEL_EPOCHS= 2000
MODEL_EPOCHS= 20000
MODEL_LR = 1.0e-1
MODEL_LABEL_NUM = len(np.unique(y_train))

class MLP(torch.nn.Module):
    def __init__(self, n_neurons):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(x_train.shape[1], n_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_neurons, MODEL_LABEL_NUM)
        self.ac2 = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        logits = self.fc2(x)
        x = self.ac2(logits)
        return x

network = MLP(MODEL_NEURONS)
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(network.parameters(), lr=MODEL_LR)


#x_train_tensor = torch.tensor(x_train).float()
x_train_tensor = masked_batch
y_train_tensor = torch.tensor(y_train)
x_test_tensor = torch.tensor(x_test).float()
y_test_tensor = torch.tensor(y_test)

for epoch in range(MODEL_EPOCHS):
    optimizer.zero_grad()
    
    preds = network(x_train_tensor)
    label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
    label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
    loss_value = loss(preds, label_onehot)
    loss_value.backward()        
    optimizer.step()

    test_preds = network.forward(x_test_tensor)        
    accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean() 
    print(f'Epoch {epoch}/{MODEL_EPOCHS} - Loss: {loss_value.item()} - Test accuracy: {accuracy}')  
    
print(accuracy.item())


Epoch 0/20000 - Loss: 0.2809506356716156 - Test accuracy: 0.41553211212158203
Epoch 1/20000 - Loss: 0.4091391861438751 - Test accuracy: 0.41093000769615173
Epoch 2/20000 - Loss: 0.7127585411071777 - Test accuracy: 0.40508151054382324
Epoch 3/20000 - Loss: 0.3721371591091156 - Test accuracy: 0.2360498607158661
Epoch 4/20000 - Loss: 0.4496758282184601 - Test accuracy: 0.06375838816165924
Epoch 5/20000 - Loss: 0.550395667552948 - Test accuracy: 0.04515819624066353
Epoch 6/20000 - Loss: 0.6978278160095215 - Test accuracy: 0.23854266107082367
Epoch 7/20000 - Loss: 0.4365551173686981 - Test accuracy: 0.2507190704345703
Epoch 8/20000 - Loss: 0.4074256718158722 - Test accuracy: 0.2804410457611084
Epoch 9/20000 - Loss: 0.34140485525131226 - Test accuracy: 0.22176414728164673
Epoch 10/20000 - Loss: 0.2877771556377411 - Test accuracy: 0.4669223427772522
Epoch 11/20000 - Loss: 0.27053889632225037 - Test accuracy: 0.46730583906173706
Epoch 12/20000 - Loss: 0.308896541595459 - Test accuracy: 0.45484

In [26]:
# Save model
torch.save(network.state_dict(), os.path.join(PROJ_DIR,'assets','models','avila-mlp-ood-zeros.pth'))